# 1. For the first trying, use 0 and 1 as output parameters
##import TensorFlow






In [ ]:

#@test {"skip": true}

# tensorflow_federated_nightly also bring in tf_nightly, which
# can causes a duplicate tensorboard install, leading to errors.
!pip uninstall --yes tensorboard tb-nightly

!pip install --quiet --upgrade tensorflow-federated-nightly
!pip install --quiet --upgrade nest-asyncio
!pip install --quiet --upgrade tb-nightly  # or tensorboard, but not both

import nest_asyncio
nest_asyncio.apply()

Uninstalling tensorboard-2.4.1:
  Successfully uninstalled tensorboard-2.4.1
     |████████████████████████████████| 604kB 5.4MB/s 
     |████████████████████████████████| 174kB 7.6MB/s 
     |████████████████████████████████| 456.8MB 35kB/s 
     |████████████████████████████████| 51kB 5.9MB/s 
     |████████████████████████████████| 194kB 35.0MB/s 
     |████████████████████████████████| 4.0MB 42.8MB/s 
     |████████████████████████████████| 5.9MB 49.9MB/s 
     |████████████████████████████████| 1.3MB 47.4MB/s 
     |████████████████████████████████| 471kB 47.4MB/s 
     |████████████████████████████████| 4.0MB 38.7MB/s 
     |████████████████████████████████| 3.9MB 38.4MB/s 
ERROR: tensorflow 2.4.1 requires tensorboard~=2.4, which is not installed.
ERROR: tensorflow 2.4.1 has requirement gast==0.3.3, but you'll have gast 0.4.0 which is incompatible.
ERROR: tensorflow 2.4.1 has requirement grpcio~=1.32.0, but you'll have grpcio 1.34.1 which is incompatible.
ERROR: tensorflow 2.4.1 

In [ ]:
%load_ext tensorboard

In [ ]:
import collections
import csv
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_federated as tff


##import sample dataset

In [ ]:
# load google drive
import os
from google.colab import drive
drive.mount('/content/drive')

path = "/content/drive/My Drive"

os.chdir(path)
os.listdir(path)

Mounted at /content/drive


['简历.gdoc', 'Colab Notebooks', 'Sample Dataset.csv', '530.ipynb']

In [ ]:
# import sample dataset
df = pd.read_csv ('Sample Dataset.csv',index_col=False)
df = df.dropna()
df['label'] = 0
feature = ['cough', 'fever', 'sore_throat', 'shortness_of_breath', 'head_ache', 
           'corona_result', 'age_60_and_above', 'gender']
df['label'] = np.where((df['lc28'] == 1) | (df['lc56'] == 1), 1,0)
del df['lc28']
del df['lc56']
df['gender'] = np.where((df['gender'] == 'male'), 1,0)
df['age_60_and_above'] = np.where((df['age_60_and_above'] == 'Yes'), 1,0)
df = df[feature + ['label']]
df_positive = df[df['label'] == 1]
df_negative = df[df['label'] == 0]
df_negative = df_negative.sample(n=(len(df_positive)))
print(len(df_positive))
print(len(df_negative))
dfs = [df_positive,df_negative]
df_balanced = pd.concat(dfs)
df_balanced = df_balanced.sample(frac=1)
del df_positive
del df_negative

train_val_split = int(len(df_balanced)*0.8)
df_train = df_balanced[:train_val_split]
df_test = df_balanced[train_val_split:]
print(len(df_train),' / ',len(df_test))

df_train

# train_label = df_train.pop('label')
# train_data = tf.data.Dataset.from_tensor_slices((df_train.values, train_label.values))


# test_label = df_test.pop('label')
# test_data = tf.data.Dataset.from_tensor_slices((df_test.values, test_label.values))






4215
4215
6744  /  1686


,cough,fever,sore_throat,shortness_of_breath,head_ache,corona_result,age_60_and_above,gender,label
29443,0,0,0,1,0,1.0,0,1,1
32110,0,0,0,0,1,1.0,1,1,1
5597,1,0,0,0,0,0.0,0,1,0
11821,1,0,1,0,0,1.0,0,1,0
61871,0,0,1,0,0,1.0,0,0,0
...,...,...,...,...,...,...,...,...,...
12004,1,1,0,0,0,1.0,0,1,0
9319,1,0,0,0,0,0.0,0,1,1
38566,1,0,0,1,1,0.0,0,0,0
71213,0,1,0,0,1,0.0,1,0,1


In [ ]:
import keras.backend as K
from keras.layers import Input, Dense
from keras.models import Model
from keras import Sequential

def getPrecision(y_true, y_pred):
    TP = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))#TP
    N = (-1)*K.sum(K.round(K.clip(y_true-K.ones_like(y_true), -1, 0)))#N
    TN=K.sum(K.round(K.clip((y_true-K.ones_like(y_true))*(y_pred-K.ones_like(y_pred)), 0, 1)))#TN
    FP=N-TN
    precision = TP / (TP + FP + K.epsilon())#TT/P
    return precision

def getRecall(y_true, y_pred):
    TP = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))#TP
    P=K.sum(K.round(K.clip(y_true, 0, 1)))
    FN = P-TP #FN=P-TP
    recall = TP / (TP + FN + K.epsilon())#TP/(TP+FN)
    return recall
    
inputs = Input(shape=(8,))
x = Dense(8, activation='relu')(inputs)
y = Dense(1, activation='sigmoid')(x)

model = Model(inputs=inputs, outputs=y)
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=False, label_smoothing=0),
      metrics=[tf.keras.metrics.BinaryAccuracy(threshold=0.5),getRecall,getPrecision])
model.summary()

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 8)]               0         
_________________________________________________________________
dense_10 (Dense)             (None, 8)                 72        
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 9         
Total params: 81
Trainable params: 81
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# df_train[feature]

# df_train['label']
history = model.fit(df_train[feature], df_train['label'], batch_size=128, epochs=100,
                        validation_data=(df_test[feature], df_test['label']), verbose = 1)



Epoch 1/100
53/53 [==============================] - 0s 3ms/step - loss: 0.6573 - binary_accuracy: 0.5946 - getRecall: 0.3125 - getPrecision: 0.7281 - val_loss: 0.6568 - val_binary_accuracy: 0.5946 - val_getRecall: 0.3328 - val_getPrecision: 0.7573
Epoch 2/100
53/53 [==============================] - 0s 2ms/step - loss: 0.6573 - binary_accuracy: 0.5946 - getRecall: 0.3053 - getPrecision: 0.7412 - val_loss: 0.6569 - val_binary_accuracy: 0.5947 - val_getRecall: 0.3328 - val_getPrecision: 0.7573
Epoch 3/100
53/53 [==============================] - 0s 2ms/step - loss: 0.6572 - binary_accuracy: 0.5947 - getRecall: 0.3080 - getPrecision: 0.7294 - val_loss: 0.6568 - val_binary_accuracy: 0.5947 - val_getRecall: 0.3385 - val_getPrecision: 0.7229
Epoch 4/100
53/53 [==============================] - 0s 2ms/step - loss: 0.6573 - binary_accuracy: 0.5947 - getRecall: 0.3080 - getPrecision: 0.7358 - val_loss: 0.6569 - val_binary_accuracy: 0.5947 - val_getRecall: 0.3385 - val_getPrecision: 0.7229
Epoc

# 2. For the second trying, use 0 ， 1（lc28） ， 2（lc56） as output parameters

In [ ]:
# import sample dataset
df = pd.read_csv ('Sample Dataset.csv',index_col=False)
df = df.dropna()

# label1=1 is no long covid, label2=1 is lc28, label3=1 is lc56, 
df['label1'] = 0
df['label2'] = 0
df['label3'] = 0
feature = ['cough', 'fever', 'sore_throat', 'shortness_of_breath', 'head_ache', 
           'corona_result', 'age_60_and_above', 'gender']
label = ['label1', 'label2', 'label3']
df['label1'] = np.where((df['lc28'] == 0) & (df['lc56'] == 0), 1,0)
df['label2'] = np.where((df['lc28'] == 1), 1,0)
df['label3'] = np.where((df['lc56'] == 1), 1,0)

del df['lc28']
del df['lc56']
df['gender'] = np.where((df['gender'] == 'male'), 1,0)
df['age_60_and_above'] = np.where((df['age_60_and_above'] == 'Yes'), 1,0)

df = df[feature + ['label1'] + ['label2'] + ['label3']]

df_1 = df[df['label1'] == 1]
df_2 = df[df['label2'] == 1]
df_3 = df[df['label3'] == 1]

df_1 = df_1.sample(n=(len(df_2)+len(df_3)))

print(len(df_1),len(df_2),len(df_3))

dfs = [df_1,df_2,df_3]
df_balanced = pd.concat(dfs)
df_new = df_balanced.sample(frac=1)
del df_1
del df_2
del df_3

train_val_split = int(len(df_new)*0.8)
df_train = df_new[:train_val_split]
df_test = df_new[train_val_split:]
print(len(df_train),' / ',len(df_test))


df_new



4511 4215 296
7217  /  1805


,cough,fever,sore_throat,shortness_of_breath,head_ache,corona_result,age_60_and_above,gender,label1,label2,label3
45265,1,0,0,0,0,0.0,0,0,1,0,0
53994,1,0,0,0,0,1.0,0,0,1,0,0
24727,1,0,0,0,0,0.0,0,1,0,1,0
2946,1,0,0,0,0,1.0,0,1,1,0,0
39800,1,0,0,0,1,0.0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
33274,1,0,0,0,0,1.0,1,1,0,1,0
59240,0,1,0,0,0,0.0,0,0,1,0,0
55436,1,0,0,0,0,1.0,0,0,1,0,0
18012,1,0,1,0,0,0.0,0,1,1,0,0


In [ ]:
from keras import optimizers
from keras.layers import Input, Dense
from keras.models import Model
from keras import metrics

inputs = Input(shape = (8,))
x = Dense(8, activation = 'relu' )(inputs)
y = Dense(3, activation = 'softmax' )(x)
sgd = optimizers.SGD(lr=0.01, clipnorm=1.)
ada = optimizers.Adagrad(lr=0.01, epsilon=None, decay=0.0)
model = Model(inputs = inputs, outputs = y)
model.compile(optimizer = ada,
      loss = 'categorical_crossentropy',
      metrics = [metrics.mae, metrics.categorical_accuracy,getRecall,getPrecision] )
model.summary()

Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, 8)]               0         
_________________________________________________________________
dense_12 (Dense)             (None, 8)                 72        
_________________________________________________________________
dense_13 (Dense)             (None, 3)                 27        
Total params: 99
Trainable params: 99
Non-trainable params: 0
_________________________________________________________________


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [ ]:

history = model.fit(df_train[feature], df_train[label], batch_size=64, epochs=100,
            validation_data=(df_test[feature], df_test[label]), verbose = 1)



Epoch 1/100
113/113 [==============================] - 1s 4ms/step - loss: 1.0862 - mean_absolute_error: 0.4242 - categorical_accuracy: 0.5026 - getRecall: 0.0692 - getPrecision: 0.5113 - val_loss: 1.0451 - val_mean_absolute_error: 0.4126 - val_categorical_accuracy: 0.4958 - val_getRecall: 0.0885 - val_getPrecision: 0.5911
Epoch 2/100
113/113 [==============================] - 0s 2ms/step - loss: 0.9986 - mean_absolute_error: 0.4022 - categorical_accuracy: 0.5045 - getRecall: 0.1017 - getPrecision: 0.5441 - val_loss: 1.0004 - val_mean_absolute_error: 0.3974 - val_categorical_accuracy: 0.5053 - val_getRecall: 0.1237 - val_getPrecision: 0.5457
Epoch 3/100
113/113 [==============================] - 0s 2ms/step - loss: 0.9645 - mean_absolute_error: 0.3900 - categorical_accuracy: 0.5153 - getRecall: 0.1481 - getPrecision: 0.5510 - val_loss: 0.9805 - val_mean_absolute_error: 0.3883 - val_categorical_accuracy: 0.5341 - val_getRecall: 0.1833 - val_getPrecision: 0.5692
Epoch 4/100
113/113 [====